<a href="https://colab.research.google.com/github/jose-mip/IBM-DS-Capstone/blob/main/Week3_assignment_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
!pip install geocoder
import geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 2.2 MB/s 


# Importing from the web
Import the data from the Wikipedia table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

I use `pd.read_html` to transform the HTML table into a dataframe.
The content of each cell is then transferred to the mapping function `getCellData` that splits the postal code, the borough and the neighbourhood name. An intermediate list (array) of dictionaries is used to create the final dataframe.

The dataframe is then cleaned by getting rid of the rows that contain undefined boroughs. 

In [4]:
origin_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Pandas already allows to parse HTML tables within a web page; # we do not need
# this information categorized in a dataframe yet, but we can use the dataframe
# to parse the table and get all the postal codes. 
# We flatten the table and apply a filter to it to clean each cell's content; 
# Postal codes are composed of three characters, which correspond to
# the first three of each cell.

# Cell processing function
def getCellData(x):
  x = x.strip()
  r = {'postalCode': x[0:3]}
  if x.find(')') > -1:
    r['Borough'] = x[3:x.index('(')]
    r['Neighbourhood'] = x[x.index('(')+1:x.index(')')].replace('/', ',').replace(' ,', ',')
    return r
  else:
    r['Borough'] = x[3::]
  return r

# From web to pandasa and applying processor function
postal_codes = pd.read_html(origin_url)[0] \
  .applymap( getCellData ) \
  .stack().values

# List of dicts to dataframe
postal_codes = pd.DataFrame.from_records(postal_codes)

# Drop rows with unassigned borough
postal_codes = postal_codes[ postal_codes['Borough'] != 'Not assigned' ]

# Set the unassigned neighbourhood to be the same as the borough
postal_codes.loc[postal_codes['Neighbourhood'].isnull(), ['Neighbourhood']] = \
postal_codes.loc[postal_codes['Neighbourhood'].isnull(), ['Borough']] 

# Shape & show
print("Shape:", postal_codes.shape)
postal_codes.head()

Shape: (103, 3)


,postalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


# Getting the geolocation code

The lat/lon location of each postal code is obtained from the geocoder module (failed) or the CSV (success).

In [6]:
# Iterate each postal code
locations = []
for pc in postal_codes['postalCode']:
  loc = None
  print("Getting location for postal code {}...".format(pc))
  while loc == None:
    g = geocoder.google('{}, Toronto, Ontario'.format(pc))
    loc = g.latlng
  locations.push({"lat": loc[0], "lon": loc[1]})

## Since geocoder did not work for me I use the next cell

Getting location for postal code M3A...


KeyboardInterrupt: ignored

In [7]:
!wget "http://cocl.us/Geospatial_data" -O pre_loc.csv
pre_loc = pd.read_csv('pre_loc.csv').rename(columns={'Postal Code': 'postalCode'})
pre_loc.head()

--2022-10-07 09:37:32--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 184.51.102.17, 184.51.102.82, 2600:1417:a000::6011:4652, ...
Connecting to cocl.us (cocl.us)|184.51.102.17|:80... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://cocl.us/geospatial_data [following]
--2022-10-07 09:37:33--  https://cocl.us/geospatial_data
Connecting to cocl.us (cocl.us)|184.51.102.17|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2022-10-07 09:37:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 74.112.186.144
Connecting to ibm.box.com (ibm.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2022-10-07 09:37:35--  https://i

,postalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Producing the dataframe with the list of neighbours and their location

The two dataframes are merged using the values of the column "postalCodes" as the matching key reference.

In [8]:
# Nice!
# Now, we merge the two dataframes
dataFrame = postal_codes.merge(pre_loc, on='postalCode')
dataFrame.head()

,postalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
